In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#The trapezoidal rule that we're going to use to estimate the integral of S^(n)(x)
def S(A):
 su = 0
 for i in range(0,len(A)-1):
    ele = (A[i]+A[i+1])/2
    su = su + ele
 return su

#Specify these parameters
D_coef = 0.9
Sigma_a = 0.066
F = 0.07 #equal to nu * Sigma_f
n = 30

#initial guesses for the width a, the only parameter you re-adjust is the width. 
a = 20


#Intermediary parameters to keep things tidy
S0 = np.ones(n-2)
k0 = 1 #initial guess of k_eff, you can keep this at 0 and it will converge regardless
kn = 0
delta = a/(n-1)
x = np.linspace(delta, a-delta, n-2)
phi = []
error = 10


#Create your main matrix A, in this case it's a tridiagonal matrix. You can create each of the diagonal separately using the np.diag() function 
#and then add them all together
d = []
u = []
l = []

for i in np.arange(1, n-1):
 element = Sigma_a+2*D_coef/delta**2
 d.append(element)
for i in np.arange(1, n-2):
 element = -D_coef/delta**2
 u.append(element)
for i in np.arange(1, n-2):
 element = -D_coef/delta**2
 l.append(element)

D = np.diag(d, 0)
U = np.diag(u, 1)
L = np.diag(l, -1)

A = D+U+L

while error > 0.000001:
  phi_new = np.linalg.solve(A,(1/k0)*S0)
  Sn = F*phi_new
  kn = k0*S(Sn)/S(S0)
  error = abs(kn-k0)
  k0 = kn
  S0 = Sn.copy()


print("The multiplication factor for a slab of width", a,"cm is equal to", k0)


The multiplication factor for a slab of width 20 cm is equal to 0.7937863639641605


In [5]:
#Same idea as the above except now we're actually going to implement Jacobi-Richardson to solve Aphi = S instead of letting linalg do the work
#For our guess in the inner loop we'll be using an array of ones (and we'll see why this is not efficient)
#The trapezoidal rule that we're going to use to estimate the integral of S^(n)(x)
def S(A):
 su = 0
 for i in range(0,len(A)-1):
    ele = (A[i]+A[i+1])/2
    su = su + ele
 return su

#Specify these parameters
D_coef = 0.9
Sigma_a = 0.066
F = 0.07 #equal to nu * Sigma_f
n = 30

#initial guesses for the width a, the only parameter you re-adjust is the width. 
a = 20


#Intermediary parameters to keep things tidy
S0 = np.ones(n-2)
k0 = 1 #initial guess of k_eff, you can keep this at 0 and it will converge regardless
kn = 0
delta = a/(n-1)
x = np.linspace(delta, a-delta, n-2)
phi = []
error = 10
error2 = 10


#Create your main matrix A, in this case it's a tridiagonal matrix. You can create each of the diagonal separately using the np.diag() function 
#and then add them all together
d = []
u = []
l = []

for i in np.arange(1, n-1):
 element = Sigma_a+2*D_coef/delta**2
 d.append(element)
for i in np.arange(1, n-2):
 element = -D_coef/delta**2
 u.append(element)
for i in np.arange(1, n-2):
 element = -D_coef/delta**2
 l.append(element)

D = np.diag(d, 0)
U = np.diag(u, 1)
L = np.diag(l, -1)

A = D+U+L

#Decomposition of A = D - B, we'll immediately construct D^(-1) = Di by simply flipping the diagonal elements
di = []

for i in np.arange(1, n-1):
 element = 1/(Sigma_a+2*D_coef/delta**2)
 di.append(element)

Di = np.diag(di, 0)

B = D-A

outeriter = 0
inneriter = 0
while error > 0.000001:
  error2 = 10
  phi_0 = np.ones(n-2)
  phi_i = np.ones(n-2)
  Sg = (1/k0)*S0
  while error2 > 0.0001:
      phi_i = Di@B@phi_0 + Di@Sg
      error2 = np.linalg.norm(phi_i-phi_0)/np.linalg.norm(phi_i)
      phi_0 = phi_i
      inneriter += 1
  Sn = F*phi_i
  kn = k0*S(Sn)/S(S0)
  error = abs(kn-k0)
  k0 = kn
  S0 = Sn.copy()
  outeriter += 1


print("The multiplication factor for a slab of width", a,"cm is equal to", k0)
print("It needed", outeriter, "outer iterations to converge with", inneriter, "inner iterations for a total of", outeriter + inneriter, "iterations")
print("This is", inneriter/outeriter,"inner iterations per outer iteration")

The multiplication factor for a slab of width 20 cm is equal to 0.790468506391002
It needed 12 outer iterations to converge with 2795 inner iterations for a total of 2807 iterations
This is 232.91666666666666 inner iterations per outer iteration


In [6]:
#Same algorithm as above, except we're going to use a better guess instead of an array of ones for the inner iteration
#We're asked to use the previous source as the next guess which is simply phi_0 = S0/F
#We'll see that this is a very cheap improvement
#The trapezoidal rule that we're going to use to estimate the integral of S^(n)(x)
def S(A):
 su = 0
 for i in range(0,len(A)-1):
    ele = (A[i]+A[i+1])/2
    su = su + ele
 return su

#Specify these parameters
D_coef = 0.9
Sigma_a = 0.066
F = 0.07 #equal to nu * Sigma_f
n = 30

#initial guesses for the width a, the only parameter you re-adjust is the width. 
a = 20


#Intermediary parameters to keep things tidy
S0 = np.ones(n-2)
k0 = 1 #initial guess of k_eff, you can keep this at 0 and it will converge regardless
kn = 0
delta = a/(n-1)
x = np.linspace(delta, a-delta, n-2)
phi = []
error = 10
error2 = 10


#Create your main matrix A, in this case it's a tridiagonal matrix. You can create each of the diagonal separately using the np.diag() function 
#and then add them all together
d = []
u = []
l = []

for i in np.arange(1, n-1):
 element = Sigma_a+2*D_coef/delta**2
 d.append(element)
for i in np.arange(1, n-2):
 element = -D_coef/delta**2
 u.append(element)
for i in np.arange(1, n-2):
 element = -D_coef/delta**2
 l.append(element)

D = np.diag(d, 0)
U = np.diag(u, 1)
L = np.diag(l, -1)

A = D+U+L

#Decomposition of A = D - B, we'll immediately construct D^(-1) = Di by simply flipping the diagonal elements
di = []

for i in np.arange(1, n-1):
 element = 1/(Sigma_a+2*D_coef/delta**2)
 di.append(element)

Di = np.diag(di, 0)

B = D-A

outeriter = 0
inneriter = 0
while error > 0.000001:
  error2 = 10
  phi_0 = S0/F
  phi_i = np.ones(n-2)
  Sg = (1/k0)*S0
  while error2 > 0.0001:
      phi_i = Di@B@phi_0 + Di@Sg
      error2 = np.linalg.norm(phi_i-phi_0)/np.linalg.norm(phi_i)
      phi_0 = phi_i
      inneriter += 1
  Sn = F*phi_i
  kn = k0*S(Sn)/S(S0)
  error = abs(kn-k0)
  k0 = kn
  S0 = Sn.copy()
  outeriter += 1


print("The multiplication factor for a slab of width", a,"cm is equal to", k0)
print("It needed", outeriter, "outer iterations to converge with", inneriter, "inner iterations for a total of", outeriter + inneriter, "iterations")
print("This is", inneriter/outeriter,"inner iterations per outer iteration")

The multiplication factor for a slab of width 20 cm is equal to 0.7937440222340215
It needed 197 outer iterations to converge with 676 inner iterations for a total of 873 iterations
This is 3.431472081218274 inner iterations per outer iteration


In [11]:
#Same algorithm as above, We've already seen that improving your guess for the next inner loop drastically reduces computing time
#Next improvement is to use a better source term i.e. source extrapolation à la successive overrelaxation
#This introduces a parameter alpha [1,2]
#The trapezoidal rule that we're going to use to estimate the integral of S^(n)(x)
def S(A):
 su = 0
 for i in range(0,len(A)-1):
    ele = (A[i]+A[i+1])/2
    su = su + ele
 return su

#Specify these parameters
D_coef = 0.9
Sigma_a = 0.066
F = 0.07 #equal to nu * Sigma_f
n = 30
alpha = 1.3

#initial guesses for the width a, the only parameter you re-adjust is the width. 
a = 20


#Intermediary parameters to keep things tidy
S0 = np.ones(n-2)
k0 = 1 #initial guess of k_eff, you can keep this at 0 and it will converge regardless
kn = 0
delta = a/(n-1)
x = np.linspace(delta, a-delta, n-2)
phi = []
error = 10
error2 = 10


#Create your main matrix A, in this case it's a tridiagonal matrix. You can create each of the diagonal separately using the np.diag() function 
#and then add them all together
d = []
u = []
l = []

for i in np.arange(1, n-1):
 element = Sigma_a+2*D_coef/delta**2
 d.append(element)
for i in np.arange(1, n-2):
 element = -D_coef/delta**2
 u.append(element)
for i in np.arange(1, n-2):
 element = -D_coef/delta**2
 l.append(element)

D = np.diag(d, 0)
U = np.diag(u, 1)
L = np.diag(l, -1)

A = D+U+L

#Decomposition of A = D - B, we'll immediately construct D^(-1) = Di by simply flipping the diagonal elements
di = []

for i in np.arange(1, n-1):
 element = 1/(Sigma_a+2*D_coef/delta**2)
 di.append(element)

Di = np.diag(di, 0)

B = D-A

outeriter = 0
inneriter = 0
while error > 0.000001:
  error2 = 10
  phi_0 = S0/F
  phi_i = np.ones(n-2)
  Sg = (1/k0)*S0
  while error2 > 0.0001:
      phi_i = Di@B@phi_0 + Di@Sg
      error2 = np.linalg.norm(phi_i-phi_0)/np.linalg.norm(phi_i)
      phi_0 = phi_i
      inneriter += 1
  Sn = F*phi_i
  kn = k0*S(Sn)/S(S0)
  Sn = S0+alpha*(F*phi_i-S0)
  error = abs(kn-k0)
  k0 = kn
  S0 = Sn.copy()
  outeriter += 1


print("The multiplication factor for a slab of width", a,"cm is equal to", k0)
print("It needed", outeriter, "outer iterations to converge with", inneriter, "inner iterations for a total of", outeriter + inneriter, "iterations")
print("This is", inneriter/outeriter,"inner iterations per outer iteration")

The multiplication factor for a slab of width 20 cm is equal to 0.7937704276429758
It needed 24 outer iterations to converge with 551 inner iterations for a total of 575 iterations
This is 22.958333333333332 inner iterations per outer iteration
